Regular Expressions
-------------------

Regular expressions (regexes or re’s) constitute an extremely powerful, flexible and concise language for matching elements in text ranging from a few characters to complex patterns. While mastering the syntax of the regular expression language does require climbing a learning curve, this learning curve is not particularly steep, and a newcomer can find herself performing useful tasks with regular expressions almost immediately. Efforts spent learning regular expressions quickly pay off--tasks that are well suited for regular expressions abound. Indeed, regular expressions are one of the most useful computer skills, and an absolutely critical tool for data scientists. 

This document will present basic regular expression syntax and cover common use cases for regular expressions: pattern matching, filtering, data extraction, and string replacement. We will present examples using grep, which we covered previously. (In case you forgot, we used grep to find lines of a text file with a given string in them.) 

In [ ]:
# The code below is written in Python to replicate the behavior of grep, the UNIX utility
# We will examine the details of how the code works in a subsequent notebook.
# For now, just execute the code, and use the function grep(regex_expression, file_name) as-is

import re

def printMatches(text, regex_expression):
    BACKGROUND_YELLOW = '\x1b[43m'
    COLOR_RESET  = "\x1b[0m"
    regex= re.compile(regex_expression)
    matches = regex.finditer(text)
    for m in matches:
        highlighted  = text[:m.start()] # the string before the regex match
        highlighted += BACKGROUND_YELLOW + text[m.start():m.end()] + COLOR_RESET 
        highlighted += text[m.end():] # the string after the regex match
        print(highlighted)

def grep(regex_expression, file_name):
    f = open(file_name, "r")
    content = f.read()
    f.close()
    for line in content.split("\n"):
        printMatches(line, regex_expression)

### NYC Restaurant Names Data

In the notebook, we will demonstrating the various regular expressions using the set of restaurant names from `/data/uniquenames.txt`.

Let's take a peek at the contents using the `head` and `tail` commands:

In [ ]:
!head -10 /data/uniquenames.txt
!echo '........' # The "echo" command just prints in the output the string that follows the command (in this case "......")
!tail -10 /data/uniquenames.txt

Now, let's see if there are any restaurants with the string 'PANO' in them:

In [ ]:
grep('PANO', "/data/uniquenames.txt")

What can we do if we want to search for something more complex than a fixed string? Regular expressions are solving exactly this problem. 

### The atoms

The simplest regular expressions are a sequence of `atoms`. An atom can be any of the following:
* single character, 
* a dot,
* a bracket expression, 
* an anchor.

#### Single character atom

A single character atom matches itself.

#### The `.` character atom

A dot atom matches any single character (except for a new line character `\n`).

Example: Using single character atoms, and the `.` atom, let's find all restaurant names that contain the characters `AB`, followed by any character (`.`) and then the character `D`:

In [ ]:
grep('AB.D', '/data/uniquenames.txt')

#### Bracket expression atom

A bracket expression (defined by square brackets []) defines a set of characters. matches only one single character that can be any of the characters defined in a set. Example: [ABL] matches either A, B, or L.

Now, let's use a bracket expression: We want to find restaurants that contain one of the letters A,B,C,X,Y,Z followed by a digit. We specify the set of letters as `[ABCXYZ]` and the set of digits as `[0123456789]`.  

In [ ]:
grep('[ABCXYZ][0123456789][0123456789]', '/data/uniquenames.txt')

##### Brackets and ranges

Instead of typing long lists of characters in a bracket expression, we can use the range character: [0-9] is equivalent to [0123456789]. Similarly [A-Z] is equivalent to [ABCDEFGHIJKLMNOPQRSTUVWXYZ]. And [D-T] is equivalent to [DEFGHIJKLMNOPQRST]. (You get the idea.) You can also combine multiple ranges: [a-e1-9] is equivalent to [abcde123456789]. Finally, you can even specify to be excluded from the set using the character (^). For example, [^0-9] matches any character other than a number.

For example, let's find restaurants that contain a letter, followed by a number, and then followed by a charather that is not a number:

In [ ]:
grep('[A-Z][0-9][^0-9]', '/data/uniquenames.txt')

Hm, we do not want to get results that have a space after the number, so let's also exclude the space character:

In [ ]:
grep('[A-Z][0-9][^0-9 ]', '/data/uniquenames.txt') 

In [ ]:
# Digit, not letter not digit not space, digit
grep('[0-9][^A-Z0-9 ][0-9]', '/data/uniquenames.txt') 

In [ ]:
# Restaurants with five digits
grep('[0-9][0-9][0-9][0-9][0-9]', '/data/uniquenames.txt') 

#### Anchor

Anchor atoms are special characters, used to define the location of a regex within a line. 

The anchor `^` specifies the *beginning of a line*, the anchor `$` specifies the end of a line. The anchor `\b` specifies the word boundary.

Example: Find restaurant names that start with the characters `BAL`

In [ ]:
grep('^BAL', '/data/uniquenames.txt')

Example: Find restaurant names that end with the characters `NORTH`

In [ ]:
grep('NORTH$', '/data/uniquenames.txt')

In [ ]:
# All restaurants that end with 4 digits
grep('[0-9][0-9][0-9][0-9]$', '/data/uniquenames.txt')

Example: Let's try to find restaurants containing the word `COLUMBIA`:

In [ ]:
grep(' COLUMBIA ', '/data/uniquenames.txt')

In [ ]:
# Notice that adding space is not sufficient
grep(' COLUMBIA ', '/data/uniquenames.txt')

Hm, something is wrong. We also get COLUMBIANO, COLUMBIANAS, and other words. We want only the word COLUMBIA, so we add the word anchors:

In [ ]:
# The r'....' is a "raw" string, and allows us to enter
# backslash without having to "escape" the backslash.
# Otherwise Python will interpret \b as a single special
# character, and not as two characters \b that are part of the regex
grep(r'\bCOLUMBIA\b', '/data/uniquenames.txt')

#### Basic Patterns

* `a, X, 9, ....`: -- ordinary characters just match themselves exactly. 
* `. ^ \$ * + ? { [ ] \ | ( )`: The **meta-characters** which do not match themselves because they have special meanings (more info below)
* `.` (a period) -- matches any single character except newline '\n'
* `\t, \n, \r`: Special characters, tab, newline, return
* `^` = start, `$` = end -- match the start or end of the string
* `\`: inhibit the "specialness" of a character. So, for example, use \. to match a period or \\ to match a slash. If you are unsure if a character has special meaning, such as '@', you can put a slash in front of it, \@, to make sure it is treated just as a character.

#### Shortcuts

A few of the bracket expressions that we discussed above occur very often. For this reason, we have shortcuts for them:

* `\d`: matches the digits: `[0-9]`.
* `\D`: matches anything but `\d`: `[^0-9]`.
* `\w`: matches any alphanumeric character plus underscore: `[A-Za-z0-9_]`.
* `\W`: matches anything but `\w`: `[^A-Za-z0-9_]`
* `\s`: matches any "whitespace" character (space, tab, newline, etc): `[ \t\n\r\f\v]`.
* `\S`: matches anything but `\s`: `[^ \t\n\r\f\v]` .
* `\b`: matches the breaks between alphanumeric and non-alphanumeric characters (an empty string), the boundary between `\w` and `\W`. Useful for ensuring that what you match is actually a word.
* `\B`: matches anything but `\b`. Useful for ensuring your match is in the middle of a word.



#### In class exercises

Write a regular expression for:

* Match any character
* Match the end of line
* Match any digit
* Find all characters that are not digits
* Find all words with four letters
* Find every line that starts with a digit
* Find all empty lines
* Find all lines with 4 characters


### Regular Expressions: Operators

#### Alternation |

The alternation operator `|` defines one or more alternatives regular expressions that need to be true for the string to match the regular expression. 

For example, if we are looking for names that contain either the word `GREEK` or the word `RUSSIAN`, we issue the following command: 

In [ ]:
grep('GREEK|RUSSIAN|FRENCH', '/data/uniquenames.txt')

#### Repetition {m,n}

A repetition operator specifies that the atom or expression immediately before the repetition may be repeated. For example, if we are looking for restaurants that contain the letter I, three to five times:  

In [ ]:
grep('I{3,5}', '/data/uniquenames.txt')

Now, let's find all the restaurants that have a name length from 50 to 55 characters:

In [ ]:
grep('^.{50,55}$', '/data/uniquenames.txt')

In the repetition operator {m,n}, we can skip putting the upper limit if we want to say, "anything with m matches and above". For example, let's find all the restaurants that have a name length 60 characters and above:

In [ ]:
grep('^.{60,}$', '/data/uniquenames.txt')

##### Repetition shortcuts (very common!): 

* `* = {0,}`. The `*` character means match the previous atom zero or more times
* `+ = {1,}`. The `+` character means match the previous atom one or more times
* `? = {0,1}`. The `*` character means match the previous atom zero or one times






Find all restaurants that start with one or more digits, followed by a space.

In [ ]:
grep('^[0-9]+ ', '/data/uniquenames.txt')

Find all restaurants that start with a letter, followed by one or more digits, followed by a space.

In [ ]:
grep('^[A-Z][0-9]+ ', '/data/uniquenames.txt')

In [ ]:
# Find all restaurants
# Beggining with one or more letters // ^[A-Z]+
# followed by one or more digits // [0-9]+
# Followed by any number of charaters // .*
# and ending with BAR  // BAR$
grep('^[A-Z]+[0-9]+.*BAR$', '/data/uniquenames.txt')

Find all restaurants that start with the word STARBUCKS, followed by any number of characters, and then have a digit.

In [ ]:
grep('STARBUCKS.*[0-9]+', '/data/uniquenames.txt')

#### Grouping ()

In the group operator, when a group of characters is enclosed in parentheses, the next operator applies to the whole group, not only the previous characters. 

For example: Find all the restaurants that start (`^`) with 8 or more repetitions (`{8,}`) of the `\w+ ` pattern (alphanumeric characters followed by space):

In [ ]:
grep(r'^(\w+ ){8,}', '/data/uniquenames.txt')

#### In class exercices

What do these regular expressions match?

* b (cd)*
* h (d)+
* j? k+
* (cd){2,5}
* o(pre){3,}
* Panos|Ipeirotis

#### In class exercises (advanced)

Write down the regular expressions for the following:

* A telephone number (e.g, 212-555-0921)
* A zip+4 code (e.g, 10012-1809)
* For matching a float number (e.g., +12.34 or -1.457 or 1023.4568)
* Dollar amount with optional cents  (e.g. \$0.33, \$784)
* Time of Day (e.g. 12:15am, 3:34pm)
* Match urls  only of the form http://www.alphanumeric.com
* Match an email of the form username@domain (assume  that the domain might be in the form alphanumeric.alphanumeric, or alphanumeric.alphanumeric.alphanumeric)   



### Group references

Sometimes it is handy to be able to refer to a match that was made earlier in a regex. This is done with **backreferences**, which refer to groups. `\k` is the backreference specifier, where `k` is a number, which refers to the `k`-th regular expression *that was enclosed in parenthesis*.

For example, find if the first character(s) of a line are the same as the last:


In [ ]:
grep(r'^(.{3,}).*\1$', '/data/uniquenames.txt')

Or find all the restaurant names that the first 5 characters (or more) are identical to the last characters.

In [ ]:
grep(r'^([A-Z]+)\1$', '/data/uniquenames.txt')

Find all names that have three consecutive same digits

In [ ]:
grep(r'([0-9])\1\1', '/data/uniquenames.txt')

As we are going to see, these backreferences will also be of tremendous use for extraction purposes.

In [ ]:
#### Naming groups
# The group that follows the term "DOUBLE" is named "doublewhat" and we can refer to it as \doublewhat
grep(r'DOUBLE (?P<doublewhat>\w+)', '/data/uniquenames.txt')


#### In class exercise (advanced)

Say that you have a file with telephone numbers written in a variety of forms: 

* 679-397-5255
* 2126660921
* 212-998-0902
* 888-888-2222
* 800-555-1211
* 800 555 1212
* 800.555.1213
* (800) 555-1214
* 1-800-555-1215
* 1(800)555-1216
* 800-555-1212-1234
* 800-555-1212x1234
* 800-555-1212 ext. 1234
* work 1-(800) 555.1212 #1234

The task is to standardize everything in the form (xxx)-xxx-xxx.


To make the process interactive, go to http://regex101.com/?#python, copy and paste the numbers above in the textarea called "Text String", and then try to write the regular expression above. (As a side note, the website provides excellent explanations about the meaning of the regular expression that you write down.) Remember to put the "g" character in the small textfield next to the regex: this has the same meaning as in sed, and it means "find globally" the regex, not just the first occurence.


If you manage to deal with that task, consider the case of also having international country calling codes (e.g., +1 for US, +44 for UK, +7 for Russia, +30 for Greece, +354 for Iceland etc), and also standardizing the extensions.

### Additional Regex Resources

* [Visual Regular Expression Tester](http://www.debuggex.com/?flavor=pcre)
* [Test Python Regular Expressions Online](http://www.pyregex.com/)
* [Regular Expressions 101](http://regex101.com/?)
* [Python's re Library Official Documentation](http://docs.python.org/2/library/re.html)
* [Regular expression reference at W3schools](http://www.w3schools.com/jsref/jsref_obj_regexp.asp)
* [Parsing phone numbers using Python and regular expressions](http://www.diveintopython.net/regular_expressions/phone_numbers.html)

### More Advanced Regular Expressions

And the ones below get a little bit more advanced:

* `*?`, `+?`: ordinarily, `*`, `+` and `?` are **greedy**. This means they are matching the longest possible string that satisfies the regular expression. Adding the `?` to any of these makes it non-greedy, instead matching the shortest possible expression. 
* `(?: )`: A non-capturing group. This works just as `()`, but doesn’t hold on to the matched contents.
* `(?<=x)`: Matches any string that is preceded by x (an arbitrary regular expression).
